# **This notebook scrapes data for Project 2**

In [1]:
from __future__ import print_function, division
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import re

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver



requests.__path__


You are using pip version 18.1, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


['/anaconda3/lib/python3.7/site-packages/requests']

In [3]:
def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #print(soup.prettify())
    return(soup)

In [4]:
def get_soup_selenium(url):
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    return(soup)

In [5]:
def flatten(list_name):
    list_name = [t for c in list_name for t in c]
    return(list_name)

In [6]:
def stripHTML(v):
    v = [re.sub(r'\t', '', x) for x in v]
    v = [re.sub(r'\r', '', x) for x in v]
    v = [re.sub(r'\n', '', x) for x in v]
    return(v)

## **I. Output (congestion)**

In [6]:
url = 'http://inrix.com/scorecard/'

In [7]:
soup = get_soup_selenium(url)

In [8]:
tables = soup.find_all("table")
len(tables)

1

In [9]:
rows = [row for row in tables[0].find_all('tr')]
len(rows)

1361

In [10]:
#rows[1]

In [11]:
tds = []
for i, row in enumerate(rows[1:]):
    td = rows[i].find_all('td')
    if td == []:
        pass
    else:
        tds.append(td)


In [12]:
tds[0][-1].text

'12%'

In [13]:
city = [tds[i][0].find_all('span')[0].text for i in range(len(tds))]
country = [tds[i][0].find_all('span')[1].text for i in range(len(tds))]
pct_time_in_traffic = [tds[i][-1].text for i in range(len(tds))]

In [14]:
len(pct_time_in_traffic)

1359

In [15]:
city[0:5]

['Los Angeles, CA',
 'Moscow',
 'New York City, NY',
 'Sao Paulo',
 'San Francisco, CA']

In [16]:
country = stripHTML(country)
city = stripHTML(city)
pct_time_in_traffic = stripHTML(pct_time_in_traffic)


In [17]:
pct_time_in_traffic = [re.sub(r'%', '', x) for x in pct_time_in_traffic]

In [18]:
df_traffic = pd.DataFrame(
    {'country': country,
    'city': city,
    'pct_time_in_traffic': pct_time_in_traffic
    })

In [19]:
df_traffic[['city','state']] = df_traffic['city'].str.split(',', expand = True)

In [20]:
df_traffic['country'] = [re.sub(r'Major City', '', x) for x in df_traffic['country']]
df_traffic['country'] = [re.sub(r'Mega City', '', x) for x in df_traffic['country']]

In [21]:
df_traffic['region'] = ''

In [25]:
df_traffic.region.value_counts()

Europe           784
North America    320
South America    192
Asia              53
Africa            10
Name: region, dtype: int64

In [23]:
df_traffic.loc[df_traffic['region'].isnull(), 'country']

Series([], Name: country, dtype: object)

In [24]:
check = ['North America', 'Europe','South America','Asia','Africa']
for i, row in df_traffic.iterrows():
    for c in check:
        TF = [c in x for x in df_traffic['country']]
        if TF[i] == True:
            df_traffic['region'][i] = c
            df_traffic['country'][i] = re.sub(c, '', df_traffic['country'][i])

In [26]:
df_traffic.head()

,country,city,pct_time_in_traffic,state,region
0,USA,Los Angeles,12,CA,North America
1,Russia,Moscow,26,None,Europe
2,USA,New York City,13,NY,North America
3,Brazil,Sao Paulo,22,None,South America
4,USA,San Francisco,12,CA,North America


In [27]:
numeric = ['pct_time_in_traffic']
for n in numeric:
    df_traffic[n] = df_traffic[n].apply(pd.to_numeric, errors = 'coerce')

In [28]:
df_traffic['pct_time_in_traffic'] = [x/100 for x in df_traffic['pct_time_in_traffic']]

In [29]:
df_traffic.dtypes

country                 object
city                    object
pct_time_in_traffic    float64
state                   object
region                  object
dtype: object

In [108]:
#df_traffic.state

In [615]:
df_traffic.to_pickle('city_traffic.pkl')

---

In [2]:
url = 'https://www.tomtom.com/en_gb/trafficindex/list?citySize=ALL&continent=NA&country=US'
#url = "http://inrix.com/scorecard/"


In [3]:
driver = webdriver.Chrome(chromedriver)
driver.get(url)

In [4]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [5]:
#soup.prettify()

In [6]:
tables = soup.find_all("table")
len(tables)

1

In [9]:
rows = [row for row in tables[0].find_all('tr')]
N = len(rows)-1

In [13]:
tds = []
#world_rank = []

rows = [row for row in tables[0].find_all('tr')] # tr tag is for rows

for j, row in enumerate(rows[1:81]):
    tds.append(row.find_all('td'))
    # RANK
    rank = [tds[i][0].contents for i in range(len(tds))]
    # WORLD RANK
    world_rank = [tds[i][1].find('div', attrs={'class': re.compile('map-city-icon map-city-rank-icon.*')}) for i in range(len(tds))]
    
    # CITY
    cities = [tds[i][3].find('a').contents for i in range(len(tds))]
    #
    # COUNTRY
    country = [tds[i][4].find('span', attrs={'class':'ng-binding'}).contents for i in range(len(tds))]
    # CONGESTION LEVEL
    congestion_level = [tds[i][5].find('span', attrs={'class':'total-congestion-value ng-binding'}).contents for i in range(len(tds))]
    
cities = [t for c in cities for t in c]
country = [k for c in country for k in c]
congestion_level = [int(p.strip('%'))/100 for g in congestion_level for p in g]
#world_rank = [int(r.strip()) for w in world_rank for r in w]
rank = [int(r) for k in rank for r in k]


In [14]:
world_ranks = []
for w in world_rank:
    try:
        w = w.contents
        w = [str(r).strip() for r in w]
        world_ranks.append(w)
    except:
        world_ranks.append([''])
len(world_ranks)

80

In [15]:
df_congest = pd.DataFrame(np.column_stack([rank, world_ranks, cities, country, congestion_level]), 
                               columns=['rank', 'world_ranking', 'city','country','congestion_level'])


In [16]:
df_congest.head()

,rank,world_ranking,city,country,congestion_level
0,1,12,Los Angeles,United States,0.45
1,2,30,San Francisco,United States,0.39
2,3,49,New York,United States,0.35
3,4,53,Seattle,United States,0.34
4,5,65,San José,United States,0.32


In [17]:
df_congest.iloc[:,0] = df_congest.iloc[:,0].apply(pd.to_numeric)
#df_congest['world_ranking'] = df_congest['world_ranking'].astype(int)
df_congest.iloc[:,4] = df_congest.iloc[:,4].apply(pd.to_numeric)
df_congest['world_ranking'] = df_congest['world_ranking'].replace(np.NaN, None).astype(int,errors='ignore')



In [18]:
df_congest['world_ranking'] = df_congest['world_ranking'].apply(pd.to_numeric, errors = 'coerce')



In [19]:
df_congest.dtypes

rank                  int64
world_ranking       float64
city                 object
country              object
congestion_level    float64
dtype: object

In [41]:
df_congest.to_pickle('city_congestion.pkl')

## **II. Features**

### i. City population

In [196]:
url = 'http://worldpopulationreview.com/us-cities/'


In [197]:
soup = get_soup(url)
tables = soup.find_all("table")
len(tables)

1

In [198]:
#soup

In [199]:
rows = [row for row in tables[0].find_all('tr')]
len(rows)

201

In [200]:
rows[1].find_all('td')[0].contents

['1']

In [202]:
tds = []
for row in rows[1:]:
    tds.append(row.find_all('td'))
    # RANK
    rank = [tds[i][0].contents for i in range(len(tds))]
    name = [tds[i][1].find('a').contents for i in range(len(tds))]
    state = [tds[i][2].find('a').contents for i in range(len(tds))]
    pop2019 = [tds[i][3].contents for i in range(len(tds))]
    pop2016 = [tds[i][4].contents for i in range(len(tds))]
    pop2010 = [tds[i][5].contents for i in range(len(tds))]
    density2019 = [tds[i][-1].contents for i in range(len(tds))]
    
#density2019

In [203]:
cols = [rank, name, state, pop2019, pop2016, density2019]
for c in cols:
    c = flatten(c)

In [204]:
name = flatten(name)

In [205]:
state = flatten(state)

In [206]:
pop2019 = flatten(pop2019)

In [207]:
pop2016 = flatten(pop2016)

In [208]:
density2019 = flatten(density2019)

In [209]:
for i, x in enumerate(density2019):
    #print(x)
    x = x.replace("/kmÂ²", "") 
    x = x.replace(',','')
    density2019[i] = int(x)

#density2019

In [210]:
for i, x in enumerate(pop2019):
    x = x.replace(',','')
    pop2019[i] = int(x)
for i, x in enumerate(pop2016):
    x = x.replace(',','')
    pop2016[i] = int(x)

In [211]:
df_pop = pd.DataFrame(np.column_stack([rank, name, state, pop2019, pop2016, density2019]), 
                               columns=['rank', 'city', 'state','pop2019','pop2016','density2019'])



In [216]:
df_pop.dtypes

rank            int64
city           object
state          object
pop2019         int64
pop2016         int64
density2019     int64
dtype: object

In [213]:
# to numeric: pop2019, pop2016, density2019

In [214]:
df_pop['rank'] = df_pop['rank'].apply(pd.to_numeric, errors = 'coerce')
df_pop['pop2019'] = df_pop['pop2019'].apply(pd.to_numeric, errors = 'coerce')
df_pop['pop2016'] = df_pop['pop2016'].apply(pd.to_numeric, errors = 'coerce')
df_pop['density2019'] = df_pop['density2019'].apply(pd.to_numeric, errors = 'coerce')

In [215]:
df_pop.to_pickle('city_poulations.pkl')

### ii. Local taxes

In [217]:
url = 'https://taxfoundation.org/sales-tax-rates-major-cities-midyear-2016/'

In [218]:
soup = get_soup(url)
tables = soup.find_all("table")
len(tables)

1

In [46]:
#soup

In [219]:
rows = [row for row in tables[0].find_all('tr')]
len(rows)

121

In [220]:
del rows[-1]

In [221]:
len(rows)

120

In [222]:
rows[1].find_all('td')[0].contents

[<strong>City</strong>]

In [223]:
tds = []
for row in rows[2:]:
    tds.append(row.find_all('td'))
tds[0][0].contents

['Chicago']

In [224]:
del rows[-1]

In [225]:
tds[116]

[<td>Portland</td>,
 <td>Oregon</td>,
 <td>0.00%</td>,
 <td>0.00%</td>,
 <td>0.00%</td>,
 <td>116</td>]

In [226]:
tds = []
for row in rows[2:]:
    tds.append(row.find_all('td'))
    # RANK
    city = [tds[i][0].contents for i in range(len(tds)-1)]
    state = [tds[i][1].contents for i in range(len(tds)-1)]
    state_tax = [tds[i][2].contents for i in range(len(tds)-1)]
    city_tax = [tds[i][3].contents for i in range(len(tds)-1)]
    total_tax = [tds[i][4].contents for i in range(len(tds)-1)]
    tax_rank = [tds[i][5].contents for i in range(len(tds)-1)]


In [227]:
cols = [city, state, state_tax, city_tax, total_tax, tax_rank]


In [228]:
city = flatten(city)
state = flatten(state)
state_tax = flatten(state_tax)
city_tax = flatten(city_tax)
total_tax = flatten(total_tax)
tax_rank = flatten(tax_rank)

In [229]:
#city = [x.replace('(.*)','') for x in city]
city = [re.sub(r'\(.*\)','', x).rstrip() for x in city]
city[0:10]

['Chicago',
 'Baton Rouge',
 'Birmingham',
 'Montgomery',
 'New Orleans',
 'Seattle',
 'Tacoma',
 'Fremont',
 'Oakland',
 'Memphis']

In [230]:
total_tax = [float(x.replace('%',''))/100 for x in total_tax]
state_tax = [float(x.replace('%',''))/100 for x in state_tax]
city_tax = [float(x.replace('%',''))/100 for x in city_tax]
tax_rank = [int(x) for x in tax_rank]

In [231]:
df_tax = pd.DataFrame(np.column_stack([city, state, state_tax, city_tax, total_tax, tax_rank]), 
        columns=['city', 'state', 'state_tax', 'city_tax', 'total_tax', 'tax_rank'])


In [234]:
df_tax.dtypes

city          object
state         object
state_tax    float64
city_tax     float64
total_tax    float64
tax_rank       int64
dtype: object

In [233]:
numeric = ['state_tax', 'city_tax', 'total_tax', 'tax_rank']
for n in numeric:
    df_tax[n] = df_tax[n].apply(pd.to_numeric, errors = 'coerce')

In [235]:
df_tax.head()

,city,state,state_tax,city_tax,total_tax,tax_rank
0,Chicago,Illinois,0.0625,0.04,0.1025,1
1,Baton Rouge,Louisiana,0.0500,0.05,0.1000,2
2,Birmingham,Alabama,0.0400,0.06,0.1000,2
3,Montgomery,Alabama,0.0400,0.06,0.1000,2
4,New Orleans,Louisiana,0.0500,0.05,0.1000,2


In [236]:
df_tax.to_pickle('city_taxes.pkl')

### iii. Walk and transit scores

In [162]:
url = 'https://www.walkscore.com/cities-and-neighborhoods/'

In [163]:
soup = get_soup(url)
tables = soup.find_all("table")
len(tables)

1

In [168]:
#soup

In [164]:
rows = [row for row in tables[0].find_all('tr')]
len(rows)

142

In [169]:
#rows

In [165]:
rows[-1]

<tr id="city_2978"> <td class="city"><a href="/AU-ACT/Canberra">Canberra</a></td> <td class="state">AU-ACT</td> <td class="score">39.9</td> <td class="tsc">--</td> <td class="bsc">--</td> <td class="pop">355,596</td> </tr>

In [171]:
rows[1].find_all('td')[0].contents

[<a href="/NY/New_York">New York</a>]

In [181]:
tds = []
for row in rows[1:]:
    tds.append(row.find_all('td'))
tds[0][0].find('a').contents

['New York']

In [182]:
tds = []
for row in rows[2:]:
    tds.append(row.find_all('td'))
    # RANK
    city = [tds[i][0].find('a').contents for i in range(len(tds))]
    state = [tds[i][1].contents for i in range(len(tds))]
    walk_score = [tds[i][2].contents for i in range(len(tds))]
    transit_score = [tds[i][3].contents for i in range(len(tds))]
    bike_score = [tds[i][4].contents for i in range(len(tds))]
    population = [tds[i][5].contents for i in range(len(tds))]


In [183]:
cols = [city, state, walk_score, transit_score, bike_score, population]


In [184]:
city = flatten(city)
state = flatten(state)
walk_score = flatten(walk_score)
transit_score = flatten(transit_score)
bike_score = flatten(bike_score)
population = flatten(population)

In [185]:
df_walkscore = pd.DataFrame(np.column_stack([city, state, walk_score, transit_score, bike_score, population]), 
        columns=['city', 'state', 'walk_score', 'transit_score', 'bike_score', 'population'])


In [190]:
df_walkscore.dtypes

city              object
state             object
walk_score       float64
transit_score    float64
bike_score       float64
population         int64
dtype: object

In [192]:
df_walkscore.tail()

,city,state,walk_score,transit_score,bike_score,population
135,Gold Coast,AU-QLD,47.8,NaN,NaN,478107
136,Sunshine Coast,AU-QLD,44.5,NaN,NaN,209263
137,Adelaide,AU-SA,53.6,NaN,NaN,1103979
138,Perth,AU-WA,50.2,NaN,NaN,1627576
139,Canberra,AU-ACT,39.9,NaN,NaN,355596


In [189]:
#city = [x.replace('(.*)','') for x in city]
df_walkscore['population'] = [x.replace(',','') for x in df_walkscore['population']]
numeric = ['walk_score', 'transit_score', 'bike_score', 'population']
for n in numeric:
    df_walkscore[n] = df_walkscore[n].apply(pd.to_numeric, errors = 'coerce')

In [193]:
df_walkscore.to_pickle('city_walkscore.pkl')

### iv. Weather

*a. Average yearly temperature*

In [32]:
url = 'https://en.wikipedia.org/wiki/List_of_cities_by_average_temperature'

In [33]:
soup = get_soup(url)
tables = soup.find_all("table")
len(tables)

6

In [248]:
#tables[3]

In [46]:
#soup

In [34]:
rows = [row for row in tables[3].find_all('tr')]
len(rows)

92

In [251]:
#rows[1]

In [35]:
rows[1].find_all('td')[0].contents

[<a href="/wiki/Antigua_and_Barbuda" title="Antigua and Barbuda">Antigua and Barbuda</a>,
 '\n']

In [36]:
tds = []
for row in rows[1:]:
    tds.append(row.find_all('td'))
tds[0][2].text

'25.4(77.7)\n'

In [37]:
tds[-2][0].text


'United States\n'

In [52]:
tds = []
for row in rows[1:]:
    tds.append(row.find_all('td'))
    # RANK
    country = [tds[i][0].text for i in range(len(tds))]
    city = [tds[i][1].text for i in range(len(tds))]
    january = [tds[i][2].text for i in range(len(tds))]
    february = [tds[i][3].text for i in range(len(tds))]
    march = [tds[i][4].text for i in range(len(tds))]
    april = [tds[i][5].text for i in range(len(tds))]
    may = [tds[i][6].text for i in range(len(tds))]
    june = [tds[i][7].text for i in range(len(tds))]
    july = [tds[i][8].text for i in range(len(tds))]
    august = [tds[i][9].text for i in range(len(tds))]
    september = [tds[i][10].text for i in range(len(tds))]
    october = [tds[i][11].text for i in range(len(tds))]
    november = [tds[i][12].text for i in range(len(tds))]
    december = [tds[i][13].text for i in range(len(tds))]
    year = [tds[i][14].text+str(i) for i in range(len(tds))]

In [27]:
city[0]


"St. John's\n"

In [58]:
year[0:5]

['26.9(80.4)\n0',
 '28.1(82.6)\n1',
 '24.6(76.3)\n2',
 '27.0(80.6)\n3',
 '26.7(80.1)\n4']

In [47]:
city[-15:-10]

['Phoenix', 'Pittsburgh', 'Portland', 'Sacramento', 'Salt LakeCity']

In [38]:
country[0:5]

['Antigua and Barbuda', 'Aruba', 'The Bahamas', 'Barbados', 'Belize']

In [71]:
country = [(x.replace('\n','')) for x in country]
city = [x.replace('\n','') for x in city]
city = [re.sub(r'\(.*\)','', x).rstrip() for x in city]
year = [x.rstrip() for x in year]

In [77]:
df_weather = pd.DataFrame(
    {'country': country,
    'city': city,
    'avg_yearly_temp': year
    })

In [78]:
df_weather[['year_C','year_F']] = df_weather['avg_yearly_temp'].str.split('(', expand = True)


In [79]:
df_weather['year_F'] = df_weather['year_F'].str.split(')', expand = True)

In [85]:
df_weather = df_weather.drop(['avg_yearly_temp'], axis = 1)

In [292]:
df_weather.head()

,country,city,year_C,year_F
0,Antigua and Barbuda,St. John's,26.9,80.4
1,Aruba,Oranjestad,28.1,82.6
2,The Bahamas,Nassau,24.6,76.3
3,Barbados,Bridgetown,27.0,80.6
4,Belize,Belize City,26.7,80.1


In [293]:
numeric = ['year_C', 'year_F']
for n in numeric:
    df_weather[n] = df_weather[n].apply(pd.to_numeric, errors = 'coerce')

In [294]:
df_weather.to_pickle('city_temp.pkl')

In [49]:
df_weather = pd.read_pickle('city_temp.pkl')

In [70]:
#list(df_weather.city[df_weather.country == 'United States'])

In [59]:
#df_weather

In [60]:
df_weather.to_pickle('city_temp.pkl')

*b. Average precipication*

In [136]:
url = 'https://www.currentresults.com/Weather/US/average-annual-precipitation-by-city.php'

In [140]:
soup = get_soup(url)


In [151]:
#soup

In [142]:
tables = soup.find_all("table")
len(tables)

3

In [185]:
#tables[0]

In [158]:
rowsall = []
for i, t in enumerate(tables):
    rows = tables[i].find_all('tr')
    rowsall.append(rows)
len(rowsall)

3

In [159]:
rowsall = flatten(rowsall)

In [176]:
len(rowsall)
len(rowsall)

54

In [177]:
rows = rowsall

In [204]:
rowsall[-1].find_all('td')

[<td>114</td>, <td>Washington, DC</td>, <td>39.7</td>, <td>1009</td>]

In [270]:
tds = []
for i, row in enumerate(rowsall[1:]):
    td = rowsall[i].find_all('td')
    if td == []:
        pass
    else:
        tds.append(td)


In [257]:
#tds.pop(0)

In [271]:
len(tds)

50

In [278]:
days = [tds[i][0].contents for i in range(len(tds))]
city = [tds[i][1].contents for i in range(len(tds))]
inches = [tds[i][2].contents for i in range(len(tds))]
mills = [tds[i][3].contents for i in range(len(tds))]

In [279]:
days = flatten(days)

In [280]:
city = flatten(city)

In [281]:
inches = flatten(inches)

In [284]:
mills = flatten(mills)

In [285]:
city[-15:-10]

['Providence, Rhode Island',
 'Raleigh, North Carolina',
 'Richmond, Virginia',
 'Riverside, California',
 'Rochester, New York']

In [286]:
df_percip = pd.DataFrame(
    {'days': days,
    'city': city,
    'percip_inches': inches,
    'percip_mil': mills
    })

In [290]:
df_percip[['city','state']] = df_percip['city'].str.split(',', expand = True)


In [295]:
numeric = ['days', 'percip_inches','percip_mil']
for n in numeric:
    df_percip[n] = df_percip[n].apply(pd.to_numeric, errors = 'coerce')

In [296]:
df_percip.dtypes

days               int64
city              object
percip_inches    float64
percip_mil         int64
state             object
dtype: object

In [297]:
df_percip.head()

,days,city,percip_inches,percip_mil,state
0,113,Atlanta,49.7,1263,Georgia
1,88,Austin,34.2,870,Texas
2,116,Baltimore,41.9,1064,Maryland
3,117,Birmingham,53.7,1364,Alabama
4,126,Boston,43.8,1112,Massachusetts


In [298]:
df_percip.to_pickle('city_percip.pkl')

## **v. Commercial density**

In [71]:
url = 'https://www.citylab.com/life/2012/09/geography-business-density/2807/'

In [72]:
soup = get_soup(url)

In [73]:
tables = soup.find_all("table")
len(tables)

1

In [74]:
rows = [row for row in tables[0].find_all('tr')]
len(rows)

21

In [75]:
len(tds)

91

In [76]:
rows[0].find_all('th')

[<th scope="col">
 				Rank</th>, <th scope="col">
 				Metro Area</th>, <th scope="col">
 				Business Establishments<br/>
 				per square mile</th>]

In [77]:
tds = []
for i, row in enumerate(rows[1:]):
    td = rows[i].find_all('td')
    if td == []:
        pass
    else:
        tds.append(td)


In [78]:
tds[1][0].text

'\r\n\t\t\t\t2'

In [79]:
rank = [tds[i][0].text for i in range(len(tds))]
city = [tds[i][1].text for i in range(len(tds))]
bus_per_sq_mi = [tds[i][2].text for i in range(len(tds))]

In [80]:
def stripHTML(v):
    v = [re.sub(r'\t', '', x) for x in v]
    v = [re.sub(r'\r', '', x) for x in v]
    v = [re.sub(r'\n', '', x) for x in v]
    return(v)

In [81]:
rank = stripHTML(rank)
city = stripHTML(city)
bus_per_sq_mi = stripHTML(bus_per_sq_mi)

In [88]:
df_busdensity = pd.DataFrame(
    {'rank': rank,
    'city': city,
    'bus_per_sq_mi': bus_per_sq_mi
    })

In [89]:
df_busdensity[['city','state']] = df_busdensity['city'].str.split(',', expand = True)

In [90]:
df_busdensity[['city1','city2','city3']] = df_busdensity['city'].str.split('-', expand = True)

In [93]:
df_busdensity.head()

,rank,bus_per_sq_mi,state,variable,value
0,1,79.0,NY-NJ-PA,city1,New York
1,2,68.7,CA,city1,Los Angeles
2,3,47.9,CA,city1,San Francisco
3,4,34.9,MA-NH,city1,Boston
4,5,33.1,FL,city1,Miami


In [92]:
#df_busdensity.unstack().reset_index()
df_busdensity = pd.melt(df_busdensity, id_vars=['rank','bus_per_sq_mi','state'], value_vars=['city1', 'city2', 'city3'])


In [94]:
df_busdensity = df_busdensity.drop(['variable'], axis = 1)

In [95]:
numeric = ['rank', 'bus_per_sq_mi']
for n in numeric:
    df_busdensity[n] = df_busdensity[n].apply(pd.to_numeric, errors = 'coerce')

In [96]:
df_busdensity = df_busdensity.drop(['rank'], axis = 1)

In [97]:
df_busdensity = df_busdensity.rename(columns={'value': 'city'})

In [98]:
df_busdensity.dtypes

bus_per_sq_mi    float64
state             object
city              object
dtype: object

In [99]:
df_busdensity.to_pickle('city_busdensity.pkl')

In [100]:
df_busdensity = pd.read_pickle('city_busdensity.pkl')

In [101]:
df_busdensity.head()

,bus_per_sq_mi,state,city
0,79.0,NY-NJ-PA,New York
1,68.7,CA,Los Angeles
2,47.9,CA,San Francisco
3,34.9,MA-NH,Boston
4,33.1,FL,Miami


In [102]:
states=df_busdensity.groupby('city')['state'].apply(list).to_dict()

In [104]:
#states

In [65]:
df_busdensity['state'] = df_busdensity['city'].map(states)

In [105]:
#list(df_busdensity.city[df_busdensity.state.isnull() == True])

## **vi. UNHabitat etc.**

In [415]:
un_data = pd.read_excel('indicator_data_US.xls')

In [417]:
un_data.indicator.value_counts()

urban_population_share_national                       177
urban_population_cities                               177
avg_annual_rate_change_percentage_urban               138
population                                             22
income_gini_coefficient_cities                         22
urban_population_countries                             22
rural_population                                       22
population_cities_by_country_1m_5m                     16
number_of_cities_1_5_million                           16
number_of_cities_500k_1_million                        16
number_of_cities_10_million_or_more                    16
urban_population_percentage_fewer_500k                 16
population_cities_by_country_10m_or_more               16
number_of_cities_5_10_million                          15
urban_population_percentage_1m_5m                      14
death_per_1000_live_births                              8
under_5_mortality_rate                                  8
infant_mortali

# loop through all states, then all cities with no data in df_xy


**Walkscore/population**

In [7]:
'https://www.walkscore.com/CT/'

'https://www.walkscore.com/CT/'

In [8]:
df_check_walk = pd.read_pickle('df_check_walk.pkl')

In [11]:
check_walk = {k: list(v) for k, v in df_check_walk.groupby('state')['city']}

len(check_walk)
#check_walk

44

In [24]:
city_check_count = sum(len(v) for v in check_walk.values())
city_check_count

226

In [13]:
for state in check_walk.keys(): 
    for city in check_walk[state]: 
        print(state,city)

AL mobile
AL huntsville
AL tuscaloosa
AL troy
AL athens
AR fort smith
AR little rock
AR pine bluff
AZ surprise
CA santa cruz
CA san rafael
CA santa barbara
CA concord
CA fairfield
CA monterey
CA dana point
CA dublin
CA escondido
CA davis
CA oceanside
CA antioch
CA santa rosa
CA watsonville
CA oxnard
CA strathearn
CA salinas
CA rohnert park
CA vallejo
CA vacaville
CA delano
CA auburn
CA lancaster
CA lodi
CO boulder
CO greeley
CO pueblo
CT stamford
CT bridgeport
CT hartford
CT waterbury
CT danbury
CT new haven
CT derby
CT new london
CT meriden
DC washington, d.c.
DE wilmington
DE dover
DE delmar
FL clearwater
FL sarasota
FL fort myers
FL melbourne
FL port st. lucie
FL panama city
FL new port richey
FL winter haven
FL ocala
FL gainesville
FL plant city
FL pensacola
FL tallahassee
FL daytona beach
FL naples
FL stuart
FL lakeland
FL cocoa
FL fort pierce
GA savannah
GA athens
GA gainesville
GA buford
GA columbus
GA augusta
GA macon
HI kaneohe
IA cedar rapids
ID boise
IL joliet
IL north chica

In [560]:
tds[1][0].text.lower()

'\nbirmingham (the largest city in alabama)\n'

In [562]:
cities

['mobile', 'huntsville', 'tuscaloosa', 'troy', 'athens']

In [30]:
state = 'NH'
soup = get_soup('https://www.walkscore.com/'+state+'/')
tables = soup.find_all("table")
rows = tables[0].find_all('tr')
tds = [row.find_all('td') for row in rows]

for city in check_walk[state]: 
    for i in range(1, len(tds)):
        if city in tds[i][0].text.lower():
            print(i, city)

9
1 manchester
2 nashua
8 portsmouth
5 rochester


# Why getting so many duplicate values????

In [127]:
states = []
cities = []
cities_df = {}
walk_score = []
transit_score = []
bike_score = []
population = []

for state in check_walk.keys(): 

    soup = get_soup('https://www.walkscore.com/'+state+'/')
    tables = soup.find_all("table")
    rows = tables[0].find_all('tr')
    tds = [row.find_all('td') for row in rows]

    for city in check_walk[state]:          
        for i in range(1, len(tds)):
            if city in tds[i][0].text.lower():
                states.append(state)
                cities.append((tds[i][0].text.lower()))
                #cities_df = dict(zip(city, tds[i][0].text.lower()))
                walk_score.append(tds[i][2].text)
                transit_score.append(tds[i][3].text)
                bike_score.append(tds[i][4].text)
                population.append(tds[i][5].text)

In [122]:
dict_all = dict(zip(cities, list(zip(states, walk_score, transit_score, bike_score, population))))
len(dict_all)

223

In [110]:
walkscore_updated = pd.DataFrame.from_dict(dict_all, orient='index').reset_index()
len(walkscore_updated)

223

In [112]:
walkscore_updated.columns = ['city', 'state', 'walk_score', 'transit_score', 'bike_score', 'population']

In [113]:
walkscore_updated['city'] = stripHTML(walkscore_updated['city'])
walkscore_updated['city'] = [x.split(' (')[0] for x in walkscore_updated['city']] 

In [115]:
walkscore_updated[walkscore_updated.state=='NH']

,city,state,walk_score,transit_score,bike_score,population
129,manchester,NH,48,--,39,"109,565"
130,nashua,NH,37,--,35,"86,494"
131,portsmouth,NH,47,--,--,"20,779"
132,rochester,NH,21,--,30,"29,752"


In [116]:
walkscore_updated['population'] = [re.sub(r',', '', x) for x in walkscore_updated['population']]

In [117]:
numeric = ['walk_score', 'transit_score', 'bike_score', 'population']
for n in numeric:
    walkscore_updated[n] = walkscore_updated[n].apply(pd.to_numeric, errors = 'coerce')

In [118]:
walkscore_updated.dtypes

city              object
state             object
walk_score         int64
transit_score    float64
bike_score       float64
population         int64
dtype: object

In [637]:
walkscore_updated

,city,state,walk_score,transit_score,bike_score,population
0,mobile,AL,33,NaN,33.0,195111
1,huntsville,AL,23,12.0,36.0,180105
2,tuscaloosa,AL,33,NaN,32.0,90468
3,troy,AL,20,NaN,NaN,18033
4,athens,AL,21,NaN,33.0,21897
5,auburn,CA,27,NaN,33.0,53380
6,athens,GA,21,NaN,33.0,21897
7,oxford,MS,10,NaN,19.0,21348
8,fort smith,AR,35,NaN,32.0,86209
9,little rock,AR,32,19.0,28.0,193524


In [119]:
walkscore_updated[walkscore_updated['city'] == 'manchester']

,city,state,walk_score,transit_score,bike_score,population
129,manchester,NH,48,NaN,39.0,109565


In [120]:
walkscore_updated.to_pickle('city_walkscores_updated.pkl')

**Temp/percipitation**

In [30]:
df_check_temp = pd.read_pickle('df_check_temp.pkl')

In [72]:
df_check_temp['state_long'] = df_check_temp['state'].map(states_abbrev)
df_check_temp['state_long'] = [sl.lower() for sl in df_check_temp['state_long']]

In [137]:
check = df_check_temp.groupby(['state_long','city']).size().reset_index()
len(check)

259

In [151]:
len(city_urls)

257

In [152]:
city_w_urls = ' ,'.join(city_urls)
#city_w_urls

In [127]:
state_urls = list(set(state_urls))
print(len(check_temps), len(state_urls), len(df_check_temp.groupby('state_long')['city']))


46 46 46


In [81]:
check_temps = {k: list(v) for k, v in df_check_temp.groupby('state_long')['city']}

check_temps

{'alabama': ['birmingham',
  'mobile',
  'huntsville',
  'tuscaloosa',
  'montgomery',
  'troy',
  'athens'],
 'arizona': ['tucson', 'surprise'],
 'arkansas': ['fort smith', 'little rock', 'pine bluff'],
 'california': ['santa cruz',
  'san rafael',
  'santa barbara',
  'concord',
  'fairfield',
  'monterey',
  'dana point',
  'dublin',
  'escondido',
  'davis',
  'oceanside',
  'antioch',
  'santa rosa',
  'watsonville',
  'oxnard',
  'strathearn',
  'salinas',
  'rohnert park',
  'vallejo',
  'vacaville',
  'stockton',
  'bakersfield',
  'delano',
  'auburn',
  'lancaster',
  'fresno',
  'lodi'],
 'colorado': ['colorado springs', 'boulder', 'greeley', 'pueblo'],
 'connecticut': ['stamford',
  'bridgeport',
  'hartford',
  'waterbury',
  'danbury',
  'new haven',
  'derby',
  'new london',
  'meriden'],
 'delaware': ['wilmington', 'dover', 'delmar'],
 'district of columbia': ['washington, d.c.'],
 'florida': ['orlando',
  'clearwater',
  'sarasota',
  'fort myers',
  'melbourne',
  'p

In [106]:
state_page_links

['https://www.facebook.com/yourweatherservice',
 'https://twitter.com/usclimatedata',
 'http://www.usclimatedata.com',
 '/climate/united-states/us',
 '/climate/north-carolina/united-states/3203',
 '#summary',
 '/climate/united-states/us',
 '/climate/north-carolina/united-states/3203',
 '/climate/charlotte/north-carolina/united-states/usnc0121',
 '#',
 '#',
 '/climate/albemarle/north-carolina/united-states/usnc0005',
 '/climate/forest-city/north-carolina/united-states/usnc0241',
 '/climate/newport/north-carolina/united-states/usnc1370',
 '/climate/andrews/north-carolina/united-states/usnc0012',
 '/climate/franklin/north-carolina/united-states/usnc0246',
 '/climate/north-wilkesboro/north-carolina/united-states/usnc0495',
 '/climate/apex/north-carolina/united-states/usnc0791',
 '/climate/gastonia/north-carolina/united-states/usnc0256',
 '/climate/ocracoke/north-carolina/united-states/usnc1001',
 '/climate/asheboro/north-carolina/united-states/usnc0021',
 '/climate/goldsboro/north-carolina

In [203]:
hrefs[20]

'/climate/california/united-states/3174'

In [49]:
url = 'https://www.usclimatedata.com/climate/united-states/us'
soup = get_soup(url)
links = soup.find_all('a')
hrefs = list([links[i]['href'] for i in range(len(links))])

In [186]:
check_temps['district of columbia'] = ['washington, d.c.','washington']
check_temps['district of columbia'] 

['washington, d.c.', 'washington']

1. Loop through main page and get each relevant state's page

In [207]:
print(len(city_states), len(city_urls_dict))

204 204


In [204]:
#state_urls = []
city_urls = []
city_urls_dict = {}
#just in case different states with same city name
city_states = {}
for h in hrefs:
    for s, c in check_temps.items():
        s_url = s.lower().replace(' ','-')
        if s_url in h:            
            #state_urls.append('https://www.usclimatedata.com'+h)
            soup = get_soup('https://www.usclimatedata.com'+h)
            for ci in c:
                
                c_url = ci.lower().replace(' ','-')

                #if c_url in list([soup.find_all('a')[i]['href'] for i in range(len(soup.find_all('a')))]):
                city_links = soup.find_all('a')
                for href in [a['href'] for a in city_links]:
                    if c_url in href:
                        city_urls.append(ci)
                        city_urls_dict[ci] = 'https://www.usclimatedata.com'+href
                        city_states[ci] = s
                        
#state_urls[0:5]


In [210]:
list(city_urls_dict.items())[0]

('birmingham',
 'https://www.usclimatedata.com/climate/birmingham/alabama/united-states/usal0054')

In [225]:
city_urls_dict

{'birmingham': 'https://www.usclimatedata.com/climate/birmingham/alabama/united-states/usal0054',
 'mobile': 'https://www.usclimatedata.com/climate/mobile/alabama/united-states/usal0899',
 'huntsville': 'https://www.usclimatedata.com/climate/huntsville/alabama/united-states/usal0826',
 'tuscaloosa': 'https://www.usclimatedata.com/climate/tuscaloosa/alabama/united-states/usal0542',
 'montgomery': 'https://www.usclimatedata.com/climate/montgomery/alabama/united-states/usal0375',
 'troy': 'https://www.usclimatedata.com/climate/troy/alabama/united-states/usal0540',
 'athens': 'https://www.usclimatedata.com/climate/athens/georgia/united-states/usga0027',
 'louisville': 'https://www.usclimatedata.com/climate/louisville/kentucky/united-states/usky1846',
 'lexington': 'https://www.usclimatedata.com/climate/lexington/kentucky/united-states/usky1079',
 'fargo': 'https://www.usclimatedata.com/climate/fargo/north-dakota/united-states/usnd0115',
 'baton rouge': 'https://www.usclimatedata.com/climat

In [235]:
city_temps = {}
city_percip = {}
for k, v in city_urls_dict.items():
    #v = list(city_urls_dict.values())[0]
    soup = get_soup(v)
    tables = soup.find_all("table")
    rows = tables[2].find_all('tr')
    for i in range(len(rows)):
        if 'Average annual precipitation' in rows[i].text:
            city_percip[k] = rows[i].text
        if 'Average temperature' in rows[i].text:
            city_temps[k] = rows[i].text
#print(rows[i].text)
#if 'precip'
#percip_inches = rows[3].text
#percip_inches

In [342]:
x = ''
def number_str(String):
    if len(String) > 0:
        index_list = []
        del index_list[:]
        for i, x in enumerate(String):
            if x.isdigit() == True:
                index_list.append(i)
        start = index_list[0]
        end = index_list[-1] + 1
        number = String[start:end]
        return (number)
number_str(x)

In [424]:
def extract_nums(col):
    col_list = list(col)
    for i, x in enumerate(col_list):
        try:
            col_list[i] = float(number_str(col_list[i]))
        except:
            col_list[i] = ''
            col_list[i] = pd.to_numeric(col_list[i], errors = 'coerce')
    return(col_list)


In [265]:
city_states = {}
for h in hrefs:
    for s, c in check_temps.items():
            for ci in c:
                city_states[ci] = s
len(city_states)
#city_states

251

In [416]:
city_state = pd.DataFrame.from_dict(city_states, orient = 'index',columns=['state_long'])

df_percip = pd.DataFrame.from_dict(city_temps, orient='index',columns=['percip_text'])

df_temp = pd.DataFrame.from_dict(city_percip, orient='index',columns=['year_F_text'])

temps_updated = pd.merge(city_state, df_percip, how = 'left', left_index=True, right_index=True)
temps_updated = pd.merge(temps_updated, df_temp, how = 'left', left_index=True, right_index=True)

In [428]:
temps_updated.tail()

,state_long,percip_text,year_F_text,percip_inches,year_F
appleton,wisconsin,Average annual precipitation - rainfall:31.05 ...,Average temperature:45.7°F,31.05,45.70
madison,wisconsin,Average annual precipitation - rainfall:34.44 ...,Average temperature:46.3°F,34.44,46.30
beloit,wisconsin,Average annual precipitation - rainfall:36.38 ...,Average temperature:47.9°F,36.38,47.90
green bay,wisconsin,Average annual precipitation - rainfall:29.53 ...,Average temperature:44.15°F,29.53,44.15
kenosha,wisconsin,Average annual precipitation - rainfall:36.14 ...,Average temperature:47.9°F,36.14,47.90


In [421]:
percip_num = list(temps_updated.percip_text)
for i, p in enumerate(percip_num):
    try:
        percip_num[i] = float(number_str(percip_num[i]))
    except:
        percip_num[i] = ''
        percip_num[i] = pd.to_numeric(percip_num[i], errors = 'coerce')
temps_updated['percip_inches'] = percip_num
len(percip_num)

251

In [426]:
temps_num = extract_nums(temps_updated.year_F_text)
temps_updated['year_F'] = temps_num

In [649]:
temps_updated = temps_updated.reset_index()

In [650]:
temps_updated.iloc[:,0].describe()

count            251
unique           251
top       strathearn
freq               1
Name: index, dtype: object

In [651]:
temps_updated.to_pickle('city_temps_updated.pkl')

In [139]:
temps_updated = pd.read_pickle('city_temps_updated.pkl')

In [151]:
temps_updated['state'] = temps_updated['state_long'].map(states_abbrev_2)

In [153]:
temps_updated.tail()

,index,state_long,percip_text,year_F_text,percip_inches,year_F,state
246,appleton,wisconsin,Average annual precipitation - rainfall:31.05 ...,Average temperature:45.7°F,31.05,45.70,WI
247,madison,wisconsin,Average annual precipitation - rainfall:34.44 ...,Average temperature:46.3°F,34.44,46.30,WI
248,beloit,wisconsin,Average annual precipitation - rainfall:36.38 ...,Average temperature:47.9°F,36.38,47.90,WI
249,green bay,wisconsin,Average annual precipitation - rainfall:29.53 ...,Average temperature:44.15°F,29.53,44.15,WI
250,kenosha,wisconsin,Average annual precipitation - rainfall:36.14 ...,Average temperature:47.9°F,36.14,47.90,WI


In [154]:
temps_updated.to_pickle('city_temps_updated.pkl')

In [135]:
states_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [148]:
states_abbrev_2 = {v : k for k, v in states_abbrev.items()}

In [150]:
states_abbrev_2 = {k.lower(): v for k, v in states_abbrev_2.items()}

states_abbrev_2

{'alaska': 'AK',
 'alabama': 'AL',
 'arkansas': 'AR',
 'american samoa': 'AS',
 'arizona': 'AZ',
 'california': 'CA',
 'colorado': 'CO',
 'connecticut': 'CT',
 'district of columbia': 'DC',
 'delaware': 'DE',
 'florida': 'FL',
 'georgia': 'GA',
 'guam': 'GU',
 'hawaii': 'HI',
 'iowa': 'IA',
 'idaho': 'ID',
 'illinois': 'IL',
 'indiana': 'IN',
 'kansas': 'KS',
 'kentucky': 'KY',
 'louisiana': 'LA',
 'massachusetts': 'MA',
 'maryland': 'MD',
 'maine': 'ME',
 'michigan': 'MI',
 'minnesota': 'MN',
 'missouri': 'MO',
 'northern mariana islands': 'MP',
 'mississippi': 'MS',
 'montana': 'MT',
 'national': 'NA',
 'north carolina': 'NC',
 'north dakota': 'ND',
 'nebraska': 'NE',
 'new hampshire': 'NH',
 'new jersey': 'NJ',
 'new mexico': 'NM',
 'nevada': 'NV',
 'new york': 'NY',
 'ohio': 'OH',
 'oklahoma': 'OK',
 'oregon': 'OR',
 'pennsylvania': 'PA',
 'puerto rico': 'PR',
 'rhode island': 'RI',
 'south carolina': 'SC',
 'south dakota': 'SD',
 'tennessee': 'TN',
 'texas': 'TX',
 'utah': 'UT',
 

---